In [1]:
from cmaqpy.prepemis import *

In [2]:
# Define the paths to data files containing the base emissions, the new emissions  
# predictions, the generation of each egu, the lookup table holding the NYISO to 
# CAMD ID conversion information, and the name for the update emissions file.
in_emis_file = '/home/jas983/models/ertac_egu/CONUS2016_S0/outputs/calc_hourly_base.csv'
co2_file = '../cmaqpy/data/ny_emis/ml_output/pred_xg_co2_fix.csv'
nox_file = '../cmaqpy/data/ny_emis/ml_output/pred_xg_nox_fix.csv'
so2_file = '../cmaqpy/data/ny_emis/ml_output/pred_xg_so2_fix.csv'
gen_file = '../cmaqpy/data/ny_emis/ed_output/thermal_with_renewable_20160805_20160815.csv'
lu_file = '../cmaqpy/data/ny_emis/ed_output/RGGI_to_NYISO.csv'
out_emis_file = '/home/jas983/models/ertac_egu/CONUS2016_S0/outputs/updated_calc_hourly_base_fix.csv'

In [3]:
# Read in the base emissions file
base_df = fmt_calc_hourly_base(base_file=in_emis_file)
# Read in ML CO2 emissions estimations
ml_co2 = fmt_like_camd(data_file=co2_file, lu_file=lu_file)
# Read in ML NOx emissions estimations
ml_nox = fmt_like_camd(data_file=nox_file, lu_file=lu_file)
# Read in ML SO2 emissions estimations
ml_so2 = fmt_like_camd(data_file=so2_file, lu_file=lu_file)
# Read in NY Simple Net generation
ed_gen = fmt_like_camd(data_file=gen_file, lu_file=lu_file)

In [4]:
ml_co2

,RGGI Facility Name,ORISPL,Unit ID,NYISO Name,PTID,2016-08-05 00:00:00,2016-08-05 01:00:00,2016-08-05 02:00:00,2016-08-05 03:00:00,2016-08-05 04:00:00,2016-08-05 05:00:00,2016-08-05 06:00:00,2016-08-05 07:00:00,2016-08-05 08:00:00,2016-08-05 09:00:00,2016-08-05 10:00:00,2016-08-05 11:00:00,2016-08-05 12:00:00,2016-08-05 13:00:00,2016-08-05 14:00:00,2016-08-05 15:00:00,2016-08-05 16:00:00,2016-08-05 17:00:00,2016-08-05 18:00:00,2016-08-05 19:00:00,2016-08-05 20:00:00,2016-08-05 21:00:00,2016-08-05 22:00:00,2016-08-05 23:00:00,2016-08-06 00:00:00,2016-08-06 01:00:00,2016-08-06 02:00:00,2016-08-06 03:00:00,2016-08-06 04:00:00,2016-08-06 05:00:00,2016-08-06 06:00:00,2016-08-06 07:00:00,2016-08-06 08:00:00,2016-08-06 09:00:00,2016-08-06 10:00:00,...,2016-08-14 08:00:00,2016-08-14 09:00:00,2016-08-14 10:00:00,2016-08-14 11:00:00,2016-08-14 12:00:00,2016-08-14 13:00:00,2016-08-14 14:00:00,2016-08-14 15:00:00,2016-08-14 16:00:00,2016-08-14 17:00:00,2016-08-14 18:00:00,2016-08-14 19:00:00,2016-08-14 20:00:00,2016-08-14 21:00:00,2016-08-14 22:00:00,2016-08-14 23:00:00,2016-08-15 00:00:00,2016-08-15 01:00:00,2016-08-15 02:00:00,2016-08-15 03:00:00,2016-08-15 04:00:00,2016-08-15 05:00:00,2016-08-15 06:00:00,2016-08-15 07:00:00,2016-08-15 08:00:00,2016-08-15 09:00:00,2016-08-15 10:00:00,2016-08-15 11:00:00,2016-08-15 12:00:00,2016-08-15 13:00:00,2016-08-15 14:00:00,2016-08-15 15:00:00,2016-08-15 16:00:00,2016-08-15 17:00:00,2016-08-15 18:00:00,2016-08-15 19:00:00,2016-08-15 20:00:00,2016-08-15 21:00:00,2016-08-15 22:00:00,2016-08-15 23:00:00
0,23rd & 3rd,7910,2301,Gowanus 5,24156,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,...,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,25.63018,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644,2.81644
1,23rd & 3rd,7910,2302,Gowanus 6,24157,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,24.601614,1.014382,24.601614,24.601614,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,...,1.014382,1.014382,1.014382,1.014382,24.601614,1.014382,1.014382,24.601614,1.014382,24.601614,24.601614,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,24.601614,1.014382,1.014382,24.601614,24.601614,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382,1.014382
2,Allegany Station No. 133,10619,1,Allegany,23514,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,30.098415,30.098415,...,6.421219,6.421219,30.098415,30.098415,30.098415,30.098415,30.098415,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,30.098415,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,6.421219,30.098415,6.421219,6.421219,6.421219,6.421219,30.098415,6.421219,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415,30.098415
3,Arthur Kill,2490,20,Arthur Kill ST 2,23512,64.47739,64.47739,64.47739,64.47739,64.47739,64.47739,64.47739,64.47739,64.47739,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,173.3391,64.47739,64.47739,64.47739,64.47739,64.47739,64.47739,64.

In [15]:
for idx in ml_co2.index:
    # Get the ORISPL and the Unit ID
    egu_orispl = ml_co2.loc[idx].ORISPL
    egu_unitid = ml_co2.loc[idx]['Unit ID']
    print(f'Working on ORISPL: {egu_orispl}\tUNIT ID:{egu_unitid}')
    # Extract this ORISPL and UNIT ID from the base DataFrame
    egu_df = base_df.loc[(base_df['orispl_code'] == egu_orispl) & (base_df['unitid'] == egu_unitid)]
    try:
        print(f'EGU Region {egu_df.iloc[0,0]}')
    except IndexError:
        continue

Working on ORISPL: 7910	UNIT ID:2301
EGU Region NYCW
Working on ORISPL: 7910	UNIT ID:2302
EGU Region NYCW
Working on ORISPL: 10619	UNIT ID:1
Working on ORISPL: 2490	UNIT ID:20
EGU Region NYCW
Working on ORISPL: 2490	UNIT ID:30
EGU Region NYCW
Working on ORISPL: 55375	UNIT ID:CT1
EGU Region NYCW
Working on ORISPL: 55375	UNIT ID:CT2
EGU Region NYCW
Working on ORISPL: 55375	UNIT ID:CT3
EGU Region NYCW
Working on ORISPL: 55375	UNIT ID:CT4
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-1A
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-1B
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-2A
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-2B
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-3A
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-3B
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-4A
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT2-4B
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT3-1A
EGU Region NYCW
Working on ORISPL: 55243	UNIT ID:CT3-1B
E

In [ ]:
# Get the name of an individual EGU -- this is how units are identified in the NY Simple Net & the ML
    

    
    # Extract the correct time window
    egu_df = egu_df.loc[base_df['datetime'].isin(ml_co2.columns[5:])]
    if len(egu_df) == 0:
        print('Warning: this unit was not found in the CAMD data... skipping')
    else:
        # Replace the CO2 emissions values
        # NOTE: this is probably a dangerous way of doing this -- might be better to add the datetime as another index in the egu_df
        egu_df['co2_mass (tons)'] = ml_co2.loc[idx, ml_co2.columns[5:]].values
        # Replace the SO2 emissions values
        egu_df['so2_mass (lbs)'] = ml_so2.loc[idx, ml_so2.columns[5:]].values
        # Replace the NOx emissions values
        egu_df['nox_mass (lbs)'] = ml_nox.loc[idx, ml_nox.columns[5:]].values
        # Replace the load values
        egu_df['gload (MW-hr)'] = ed_gen.loc[idx, ed_gen.columns[5:]].values
        # Combine this new unit data back into the base_df
        base_df.update(egu_df)


In [ ]:
# Save the updated emissions to a new CSV 
# (after dropping the datetime column that we added)
base_df = base_df.drop(columns=['datetime'])